In [1]:
import gpxpy
import psycopg2

In [2]:
filename = '11398719881.gpx'

In [3]:
with open('../data/'+ filename, 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

In [4]:
gpx_points = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            gpx_points.append((point.latitude, point.longitude))

In [8]:
conn = psycopg2.connect(
    host="localhost",
    database="gpx",
    user="root",
    password="root"
)

In [9]:
cursor = conn.cursor()

In [10]:
create_table_query = """
DROP TABLE IF EXISTS gpx_points;

CREATE TABLE IF NOT EXISTS gpx_points (
    id SERIAL PRIMARY KEY,
    filename TEXT,
    latitude DOUBLE PRECISION,
    longitude DOUBLE PRECISION,
    location GEOGRAPHY(POINT, 4326)
);
"""
cursor.execute(create_table_query)

In [11]:
try:
    for point in gpx_points:
        latitude, longitude = point
        cursor.execute("""
            INSERT INTO gpx_points (filename, latitude, longitude, location)
            VALUES (%s, %s, %s, ST_SetSRID(ST_MakePoint(%s, %s), 4326))
        """, (filename, latitude, longitude, longitude, latitude))      
    conn.commit()
except:
    conn.rollback()